# Scrape the NASA Website

In [1]:
#import dependencies

#beautiful soup
from bs4 import BeautifulSoup as bs

#requests
import requests

#splinter
from splinter import Browser

#pandas
import pandas as pd

In [2]:
#define URLs for our various news sources

url_news = "https://mars.nasa.gov/news/"
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_weather = "https://twitter.com/marswxreport?lang=en"
url_facts = "https://space-facts.com/mars/"
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [ ]:
#Pull the information for NASA's Mars News.
    #the div class for the headlines is "content_title"
    #the div class for the paragraph is "rollover_description_inner"

response = requests.get(url_news)

soup = bs(response.text, 'html.parser')

print(soup.prettify())



In [ ]:
#store the headline and text as a variable

news_headline = soup.find('div', class_="content_title").text
news_headline = news_headline.replace("\n","")

news_headline

In [ ]:
news_byline = soup.find('div', class_="rollover_description_inner").text
news_byline = news_byline.replace("\n","")
news_byline

In [ ]:
response = requests.get(url_image)
soup = bs(response.text, 'html.parser')

print(soup.prettify())

In [ ]:
#Pull the JPL Mars Image URL (using Splinter per directions in the ReadMe)

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url_image)

html = browser.html
soup = bs(html, 'html.parser')

slide = soup.find('li', class_="slide")
fancybox = slide.find('a')
fimage_name = fancybox['data-title']
fimage_desc = fancybox['data-description']
fimage_purl = fancybox['data-fancybox-href']

fimage_url = f"jpl.nasa.gov{fimage_purl}"

print(fimage_name)
print(fimage_desc)
print(fimage_url)


In [4]:
#pull the martian weather report from Twitter to find out whether Chicago is colder than mars right now
    #(it happens.)

# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)
    
# browser.visit(url_weather)

# html = browser.html
# soup = bs(html, 'html.parser')

# #tweet = soup.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0").get_text()

# print(soup.prettify())


response = requests.get(url_weather)

soup = bs(response.text, 'html.parser')

results = soup.find_all('div', string_="InSight")

results



[]

In [ ]:
#pull the list of mars facts from the mars facts website

facts = pd.read_html(url_facts)
facts = facts[0]

facts

In [ ]:
facts_html = facts.to_html()

facts_html

In [35]:
#here come the four martian hemispheres

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
    
hemisphere_urls = []
    
browser.visit(url_hemispheres)

html = browser.html
soup = bs(html, 'html.parser')

results = soup.find('div', class_='container').find_all('h3')

for result in results:
    
    temp_dict = {}
    
    print(result)
    title = str(result)
    title = title.replace("<h3>","")
    title = title.replace("</h3>","")
    print(title)
    
    browser.click_link_by_partial_text(title)
  
    html = browser.html
    soup = bs(html, 'html.parser')

    spot = soup.find('div', class_='container')
    image = spot.find('img', class_='wide-image')
    url = image['src']
    url = f"astrogeology.usgs.gov{url}"
    print(url)
    
    key1 = "title"
    key2 = "img_url"
    
    temp_dict.update({key1: title, key2: url})
    
    hemisphere_urls.append(temp_dict)
    
    browser.back()
    
print(hemisphere_urls)

<h3>Cerberus Hemisphere Enhanced</h3>
Cerberus Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
<h3>Schiaparelli Hemisphere Enhanced</h3>
Schiaparelli Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
<h3>Syrtis Major Hemisphere Enhanced</h3>
Syrtis Major Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
<h3>Valles Marineris Hemisphere Enhanced</h3>
Valles Marineris Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schia